In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the L2-SVM loss function
class L2SVM(nn.Module):
    def __init__(self, C):
        super(L2SVM, self).__init__()
        self.C = C

    def forward(self, input, target):
        scores = input
        correct_scores = scores.gather(1, target.view(-1, 1)).squeeze()
        margins = torch.clamp(scores - correct_scores.view(-1, 1) + 1, min=0)
        margins[target == -1] = 0
        loss = margins.sum() * self.C
        return loss

# Define the ConvNet
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 3072)
        self.fc2 = nn.Linear(3072, 10)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor())

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

100%|██████████| 170498071/170498071 [00:01<00:00, 100322709.79it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Initialize the model and loss function
model = ConvNet()
criterion = L2SVM(C=0.1)

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.0005)

# Train the model
num_epochs = 25

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_acc += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print('Epoch: {} \t Train Loss: {:.6f} \t Train Acc: {:.6f}'.format(epoch+1, train_loss, train_acc))

Epoch: 1 	 Train Loss: 0.610031 	 Train Acc: 0.331220
Epoch: 2 	 Train Loss: 0.420971 	 Train Acc: 0.473920
Epoch: 3 	 Train Loss: 0.367801 	 Train Acc: 0.532040
Epoch: 4 	 Train Loss: 0.332594 	 Train Acc: 0.572480
Epoch: 5 	 Train Loss: 0.302932 	 Train Acc: 0.608260
Epoch: 6 	 Train Loss: 0.279283 	 Train Acc: 0.639640
Epoch: 7 	 Train Loss: 0.257916 	 Train Acc: 0.669580
Epoch: 8 	 Train Loss: 0.238465 	 Train Acc: 0.697760
Epoch: 9 	 Train Loss: 0.220720 	 Train Acc: 0.725420
Epoch: 10 	 Train Loss: 0.203562 	 Train Acc: 0.749700
Epoch: 11 	 Train Loss: 0.186643 	 Train Acc: 0.779600
Epoch: 12 	 Train Loss: 0.172076 	 Train Acc: 0.809220
Epoch: 13 	 Train Loss: 0.160767 	 Train Acc: 0.833700
Epoch: 14 	 Train Loss: 0.149556 	 Train Acc: 0.861320
Epoch: 15 	 Train Loss: 0.141085 	 Train Acc: 0.882540
Epoch: 16 	 Train Loss: 0.132993 	 Train Acc: 0.903800
Epoch: 17 	 Train Loss: 0.126861 	 Train Acc: 0.923180
Epoch: 18 	 Train Loss: 0.121836 	 Train Acc: 0.937200
Epoch: 19 	 Train L

In [ ]:
# Save the model
PATH = '/content/drive/My Drive/COMP_551/cifa10_svm.pth'
torch.save(model.state_dict(), PATH)


In [ ]:
# Load the saved model
PATH = 'model_cifar10_svm.pth'
model = ConvNet()
model.load_state_dict(torch.load('/content/drive/My Drive/COMP_551/cifa10_svm.pth'))
model.eval()

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=3072, bias=True)
  (fc2): Linear(in_features=3072, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
test_loss = 0
test_acc = 0

with torch.no_grad(): # Disable gradient computation
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        test_acc += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
test_acc /= len(test_loader.dataset)

print('Test Loss: {:.6f} \t Test Acc: {:.6f}'.format(test_loss, test_acc))


Test Loss: 0.266906 	 Test Acc: 0.704500
